This Notebook is a brief sketch of how to use Grover's algorithm.
We start by declaring all necessary imports.

In [1]:
from pyquil.api import SyncConnection
from itertools import product
from mock import patch

import numpy as np

from grove.amplification.grover import Grover

Grover's algorithm can be used to amplify the probability of an oracle-selected bitstring in an input superposition state.

In [2]:
target_bitstring = '010'
bit = ("0", "1")
bitstring_map = {}
# We construct the bitmap for the oracle
for bitstring in product(bit, repeat=3):
    bitstring = "".join(bitstring)
    if bitstring == target_bitstring:
        bitstring_map[bitstring] = -1
    else:
        bitstring_map[bitstring] = 1

To use Grover's algorithm on quantum hardware we need to define a connection to a QVM or QPU. However we don't have a real connection in this notebook, so we just mock out the response. If you intend to run this notebook on a QVM or QPU, ensure to replace `cxn` with a pyQuil connection object.

In [3]:
with patch("pyquil.api.SyncConnection") as cxn:
    # Need to mock multiple returns as an iterable
    cxn.run_and_measure.return_value = (np.asarray([0, 1, 0], dtype=int), )

Now let's run Simon's algorithm. We instantiate the Simon object and then call its `find_mask` mehtod with the connection object and the 2-to-1 function whose mask we wish to find.
Finally we assert its correctness by checking with the mask we used to generate the function

In [5]:
grover = Grover()
found_bistring = grover.find_bitstring(cxn, bitstring_map)
assert found_bistring == target_bitstring, "Found bitstring is not the expected bitstring"

AssertionError: Found bitstring is not the expected bitstring

In [6]:
found_bistring

'[0 1 0]'

If the assertion succeeded we know we found the mask and we got our result.